#TensorFlowによる畳み込みニューラルネットワーク

TensorFlowでニューラルネットワークのプログラムを実装する方法として，セッションを準備して実行する方法，Estimatorを利用する方法，Eagerを利用する方法があります．ここでは，まずセッションを準備する方法について説明します．

必要なパッケージをインポートします．
また，GPUが利用可能かを確認します．

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.test.gpu_device_name()

'/device:GPU:0'

MNISTデータセットをダウンロードします．以下を実行するとWARNINGがいくつか表示されますが，問題ありません．

In [2]:
mnist = input_data.read_data_sets("./")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


パラメータ設定を行います．学習率(learning_rate)，学習更新回数(num_steps)，ミニバッチサイズ(batch_size)，画面出力するステップ数(display_step)を用意します．
また，ニューラルネットワークのパラメータとして，中間層のユニット数(n_hidden_1とn_hidden_2)を256とします．入力層のユニット数(num_input)は784，出力層のユニット数は(num_classes)は10とします．

In [0]:
learning_rate = 0.1
num_steps = 5000
batch_size = 100
display_step = 100

num_input = 784
fc1_units = 1024 
num_classes = 10

畳み込みニューラルネットワークを構成する畳み込み層，プーリング層を用意します．また，ネットワークのパラメータを初期化するためのweight_variableとbias_variableも用意します．

In [0]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

ネットワークモデルをcnnとして定義します．入力層(x_image)は，reshape関数を利用してサイズを28x28x1とします．最初の-1はミニバッチサイズで不定なので-1としています．次に，畳み込み層のフィルタサイズを5x5x1，フィルタ数を32としてパラメータを用意します．またバイアスもフィルタ数だけ用意します．これらをconv2dの引数に与えて畳み込み演算を行い，バイアス項を加算した値を活性化関数reluに与えてh_conv1を求めます．プーリング層は，max_pool_2x2にh_conv1を引数として与えて最大値プーリングを行いh_pool1を求めます．２層目の畳み込み層とプーリング層も同様に行います．全結合層は，入力が7x7x64，出力が1024ユニットとして，初期化を行なった後，h_fc1を求めます．また，dropoutもここで定義します．出力層は入力が1024，出力が10ユニットとなっています．算出した各クラスの値y_convを戻り値とします．


In [0]:
def cnn(x):
  with tf.name_scope('reshape'):
      x_image = tf.reshape(x, [-1, 28, 28, 1])

  with tf.name_scope('conv1'):
      W_conv1 = weight_variable([5, 5, 1, 32])
      b_conv1 = bias_variable([32])
      h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  with tf.name_scope('pool1'):
      h_pool1 = max_pool_2x2(h_conv1)

  with tf.name_scope('conv2'):
      W_conv2 = weight_variable([5, 5, 32, 64])
      b_conv2 = bias_variable([64])
      h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  with tf.name_scope('pool2'):
      h_pool2 = max_pool_2x2(h_conv2)

  with tf.name_scope('fc1'):
      W_fc1 = weight_variable([7 * 7 * 64, fc1_units])
      b_fc1 = bias_variable([fc1_units])

      h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
      h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  with tf.name_scope('dropout'):
      keep_prob = tf.placeholder(tf.float32)
      h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  with tf.name_scope('fc2'):
      W_fc2 = weight_variable([fc1_units, num_classes])
      b_fc2 = bias_variable([num_classes])
      y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
      
  return y_conv, keep_prob


学習データと教師ラベルを準備します．
placeholderは，データを格納する入れ物です．ここでは，入れ物のサイズだけを決めて，具体的な値は実行する時に与えます．

In [0]:
x = tf.placeholder(tf.float32, [None, num_input])
y_ = tf.placeholder(tf.int64, [None])


ネットワークモデルを定義します．

In [0]:
 y_conv, keep_prob = cnn(x)

誤差関数と最適化の方法を定義します．ここでは，物体認識を行うのでソフトマックスクロスエントロピーを誤差関数とします．最適化の方法には，Adamを利用します．誤差はminimizeで最小化するようにします．

In [0]:
cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

次に評価を行う準備をします．ネットワークからの出力値が最大となるクラスの番号と教師データが一致するかをequalで判定します．全てのデータに対する結果をまとめてaccuracyを算出します．

In [0]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), y_)
correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

学習を実行するために，セッションをtf.Session()で準備します．そして，セッションをsess.runで実行しますが，最初にネットワークのパラメータの初期化を行います．for文内で繰り返し学習を行います．ここでは，まずミニバッチサイズ分の学習データと教師データを取得し，sess.run(XXX)で引数に指定したtrain_opを実行します．fee_dictとして与えたデータをネットワークに順伝播し，誤差を算出します．もし，更新回数がdisplay_stepで割り切れる場合は誤差と精度を画面に出力します．

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        print(batch_x.shape, batch_y.shape)
        sess.run(train_step, feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    test_accuracy = 0
    batch_num = int(mnist.test.num_examples / batch_size)
    for i in range(batch_num):
        batch = mnist.test.next_batch(batch_size)
        test_accuracy += sess.run(accuracy, feed_dict={x: batch[0],   y_: batch[1], keep_prob: 1.0})

    test_accuracy /= batch_size
    print("Test Accuracy: %g"%test_accuracy)


(100, 784) (100,)
Step 1, Minibatch Loss= 8.8325, Training Accuracy= 0.190
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 784) (100,)
(100, 7

KeyboardInterrupt: ignored